In [87]:
import os
os.chdir("..")

In [88]:
import django
# In case that we need it later
# from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE',
'mimuse.settings')
# This is for async, in case we will see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


In [89]:
import json
import http.client
from songs.models import Song, Album
from artists.models import Artist
from datetime import datetime

In [90]:
Song.objects.all().delete()

(34, {'artists.Artist_songs': 14, 'songs.Song': 20})

In [91]:
import http.client

conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "82ce6fe0f2msh1b1cfd285dbcc80p1b3260jsn6c515c923b86",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

conn.request("GET", "/recommendations/?limit=20&seed_tracks=38MKW2tQHtyO8djIOKlEFF", headers=headers)

res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))

# print(data.decode("utf-8"))

In [92]:
song_list = list()

track = data['tracks']
for song in data['tracks']:
    # print(song['album']['release_date'])
    if(song['album']['release_date_precision']=="day"):
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], release_date= song['album']['release_date'], popularity=song['popularity']))
    else:
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], popularity=song['popularity']))
    # print (song)
# print(song_list)
bulk = Song.objects.bulk_create(song_list)

In [93]:
for song in data['tracks']:
    print(song)
    id = str(song['artists'][0]['id'])
    song_id = song['id']
    print(song_id)
    if Artist.objects.filter(api_id=id).exists():
        print('Developer msg: data already exists in database')
        continue
    else:
        conn.request("GET", "/artists/?ids={id}".format(
                        id=id), headers=headers)
        res = conn.getresponse()
        datas = json.loads(res.read().decode('utf-8'))['artists'][0]
        artist = Artist(api_id=id, name = datas['name'], popularity=datas['popularity'], image=datas['images'][0]['url'], followers=datas['followers']['total'])
        artist.save()
        song_id = song['id']
        this_song = Song.objects.get(api_id=song_id)
        print(this_song)
        artist.songs.add(this_song)
        artist.save()



{'album': {'album_type': 'ALBUM', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/41MozSoPIsD1dJM0CLPjZF'}, 'id': '41MozSoPIsD1dJM0CLPjZF', 'name': 'BLACKPINK', 'type': 'artist', 'uri': 'spotify:artist:41MozSoPIsD1dJM0CLPjZF'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PL', 'PT', 'SK', 'ES', 'SE', 'CH', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'RO', 'ZA', 'EG', 'MA', 'DZ', 'TN', 'BY', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'BS', 'BB', 'BZ', 'BW', 'BF', 'CV', 'CW', 'DM', 'GM', 'GD', 'GW', 'GY', 'HT', 'JM', 'LS', 'LR', 'MW', 'ML', 'NA', 'NE', 'SM', 'ST', 'SN', 'SC', 'SL', 'KN', 'LC', 'VC', 'SR', 'TT', 'BI', 'CM', 'TD', 'KM', 'GQ', 'SZ', 'GA', 'GN', 'MR', 'RW', 'TG', 'ZW', 'BJ', 'MG', 'MU', 'MZ', 'AO', 'C

In [95]:
for artist in Artist.objects.all():
    print(artist.songs)

songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None
songs.Song.None


In [110]:
## get artist singles
for artist in Artist.objects.all():
    api_id = artist.api_id

    conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

    headers = {
        'X-RapidAPI-Key': "82ce6fe0f2msh1b1cfd285dbcc80p1b3260jsn6c515c923b86",
        'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
    }

    conn.request("GET", "/artist_singles/?id={api_id}".format(api_id=api_id), headers=headers)
    print(api_id)
    res = conn.getresponse()
    singles = json.loads(res.read().decode('utf-8'))['data']['artist']['discography']['singles']['items']
    for single in singles: 
        if single['releases']['items'][0]['type'] == 'SINGLE':
            id = single['releases']['items'][0]['id']
            if Song.objects.filter(api_id=id).exists():
                continue
            else:
                print(id)
                song_record = Song(api_id=id)
                try:
                    conn.request("GET", "/tracks/?ids=id".format(id=id), headers=headers)

                    res = conn.getresponse()
                    song_info = json.loads(res.read().decode("utf-8"))
                    print(song_info)
                    Song.objects.filter(api_id=id).update(title=song_info['name'], popularity=song_info['popularity'], preview_url=song_info['preview_url'] )
                    song_record.save()
                    song_artist = Artist.objects.get(api_id=api_id)
                    song_artist.songs.add(song_record)
                    song_artist.save()
                except:
                    continue
        else:
            continue


2AfmfGFbe0A0WsTYm0SDTx
62lN1efwf3MajWlzsSlYFN
{'tracks': [None]}
3T6Xy6iJPpYmawK2LmJWFF
{'tracks': [None]}
6khD0lw8LFHvPWoRfhAfym
{'tracks': [None]}
16UgZxoo4YMJIJItRxi7qb
{'tracks': [None]}
2R8RsIMCdiNxkQVD96kDDW
{'tracks': [None]}
6sxMmlXY5Sx0JM48RaReuO
{'tracks': [None]}
1M1d5bzsGhY2JOVD2AU29G
{'tracks': [None]}
23yd1cYlPgl643vE8bb0WD
{'tracks': [None]}
7C8nskYbHG7N0LDrNVvt7x
{'tracks': [None]}
0Tl10vqhla5fiiiiTATT4A
{'tracks': [None]}
0NC6QFvAc9H9r5iov9QwjK
{'tracks': [None]}
03TbZLPSRsHWX11umzK4qf
{'tracks': [None]}
0tM1Pr6hTKBNJEyLSft1Fs
{'tracks': [None]}
75ii7SGF9jbD1ireJ0CeNI
{'tracks': [None]}
6cdC1cwqh3eJAXaxXJt2jv
5oe889A49PL07oAKDneshY
{'tracks': [None]}
1nu4yFtuXNSumMMUdOTq3y
{'tracks': [None]}
3Yods7fVQ2rKdzStEIJgQo
{'tracks': [None]}
4Jo48MMNKDCEDZB7K3nA8c
{'tracks': [None]}
5SWsYRZgBl6n8Z5RUeYt1M
{'tracks': [None]}
7ifIoZOqgHTXsY6bxh7y0u
{'tracks': [None]}
5EshYYiAPgdUAA1gsYA2UH
{'tracks': [None]}
5Sm5V8ybIelCv1WFliqE6X
{'tracks': [None]}
7tF6wFDR2fbXMGaK1RpAKe
{'track

KeyError: 'data'

In [109]:
print(single['releases']['items'][0]['type'] == 'SINGLE')

True


In [101]:
Song.objects.get(api_id='20iyYKtw1KjMRsMZWMPpVY')

DoesNotExist: Song matching query does not exist.